In [6]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import numpy as np
from scipy.stats import nbinom
import scipy.stats as stats
from scipy.special import gamma
import pandas as pd
import random
from math import comb
import math
from pickle import dump
from tqdm import tqdm
from pickle import load
import timeit


In [7]:
net_filename = "networks/grid.pkl"
T=365
with open((net_filename), 'rb') as f1:
        G = load(f1)
no_wild = {key:[False for i in range(T)] for key in G.nodes}  

In [58]:
def simulation4(G, RNG, pop, T, f, tau, Td, Tc, Tdz, detected_edge_weight):
    """
    n - number of premises
    T - end time of simulation
    f - transmission force function
    Td - time until detection
    Tc - time until removal
    
    
    """
    
    if RNG[0] == RNG[1] == no_wild:
        return [0]*T,{"Wild":0,"N2N":0},[],[], {"S":[0]*T,"I":[0]*T,"D":[0]*T,"C":[0]*T}
    
    else:


        active_cases_data = []
        new_cases_data = []
        wild_infections = []
        first10 = []
        compartments = {"S":[],"I":[],"D":[],"C":[]}
        S = len(G.nodes())
        I = 0
        D = 0
        C = 0


        N = len(G.nodes)
        #First Infection
        #first_infection = random.choice(list(G.nodes()))

        #G.nodes[first_infection]["status"] = ["I"] 
        #infected_premises = [first_infection]
        infected_premises = []

        #Generate Random number arrays
        check_eps_infected = RNG[0]
        check_seasonal_infected = RNG[1]

        add_premise_attr(G,pop,T)


        for n in G.nodes:
            status = G.nodes[n]["status"] 
            for s in status:
                if s != "S":
                    print(s)



        t = 0
        wild = 0
        N2N = 0

        while (t<T) :

            new_infected = []



            #Wild bird Infections
            for node in G.nodes:
                if G.nodes[node]["status"][t] == "S":
                    if check_seasonal_infected[node][t] or check_eps_infected[node][t]:
                        if N2N+wild<10:
                            first10.append(node)
                        wild+=1
                     
                        new_infected.append(node)
                        wild_infections.append(node)




            for nodeI in infected_premises:
                for nodeS in G.neighbors(nodeI):
                    if G.nodes[nodeS]["status"][t] == "S":
                        rng = np.random.uniform(0,1)
                        if (check_N2N_trans(tau,nodeS,nodeI,f,G)>rng) and (nodeS not in new_infected):
                            new_infected.append(nodeS)
                            if N2N+wild<10:
                                first10.append(nodeS)
                            N2N+=1
                            





            for n in G.nodes:
                if G.nodes[n]["zone"]:

                    if n in new_infected:
                        infected_premises.append(n)
                        G.nodes[n]["status"][t+1] = "I"
                        I+=1
                        S = S-1

                    elif G.nodes[n]["status"][t+1-Tdz:t+1] == Tdz*["I"]:
                        if t>=Td:
                            G.nodes[n]["status"][t+1] = "D"
                            detection_PZ(G,n,detected_edge_weight)
                            D+=1
                            I = I-1

                    elif G.nodes[n]["status"][t+1-Tc:t+1] == Tc*["D"]:
                        if t>=Tc:
                            G.nodes[n]["status"][t+1] = "C"
                            infected_premises.remove(n)
                            C+=1
                            D = D -1

                    else:
                        G.nodes[n]["status"][t+1] = G.nodes[n]["status"][t]

                else:

                    if n in new_infected:
                        infected_premises.append(n)
                        G.nodes[n]["status"][t+1] = "I"
                        I+=1
                        S = S-1

                    elif G.nodes[n]["status"][t+1-Td:t+1] == Td*["I"]:
                        if t>=Td:
                            G.nodes[n]["status"][t+1] = "D"
                            detection_PZ(G,n,detected_edge_weight)
                            D+=1
                            I = I-1

                    elif G.nodes[n]["status"][t+1-Tc:t+1] == Tc*["D"]:
                        if t>=Tc:
                            G.nodes[n]["status"][t+1] = "C"
                            infected_premises.remove(n)
                            C+=1
                            D = D -1

                    else:
                        G.nodes[n]["status"][t+1] = G.nodes[n]["status"][t]



            t+=1
            active_cases_data.append(len(infected_premises))
            new_cases=len(new_infected)
            new_cases_data.append(new_cases)
            compartments["S"].append(S)
            compartments["I"].append(I)
            compartments["D"].append(D)
            compartments["C"].append(C)
            





        return new_cases_data,{"Wild":wild,"N2N":N2N},wild_infections,first10,compartments

In [59]:
def add_premise_attr(G,pop,T):
    node_attr = {}
    i=0
    for node in G.nodes():
        node_attr[node] = {"pop": pop[i],
                          "status":["S" for i in range(T+1)],
                          "zone" : False}
        i+=1
        
    for e in G.edges:
        G.get_edge_data(e[1],e[0])["weight"] = 1
    nx.set_node_attributes(G,node_attr)

In [60]:
def check_N2N_trans(tau,nodeS,nodeI,f,G):
    popS = G.nodes[nodeS]["pop"]
    popI = G.nodes[nodeI]["pop"]
    force = tau * f(popI,popS) * G.get_edge_data(nodeI,nodeS)["weight"] 
    p = -expm1(-force)
    return p

In [61]:
def detection_PZ(G,node,new_weight):
    G.nodes[node]["zone"] = True
    for e in G.edges(node):
        G.get_edge_data(e[1],e[0])["weight"] = new_weight   
            
            
    for n1 in G.neighbors(node):
        G.nodes[n1]["zone"] = True
        for e in G.edges(n1):
            G.get_edge_data(e[1],e[0])["weight"] = new_weight   

In [62]:
def detection_edge_change(G,node,new_weight):
    for e in G.edges(node):
        G.get_edge_data(e[1],e[0])["weight"] = new_weight   

In [63]:
def reset_node_status(G):
    for n in G.nodes():
        nx.set_node_attributes(G,{n:{"status":["S"]}})

In [64]:
def expm1(x):
    if abs(x) < 1e-5:
        return x + 0.5*x*x
    else:
        return math.exp(x) - 1.0

In [123]:
theta = {"grid":
         {"pre2021":{"tau":0.00110078125,"eps_r":0,"eps_s":3.326e-6},
          "post2021":{"tau":0.00110078125,"eps_r": 4.5714e-05,"eps_s": 4e-05}},
         "D=1":
         {"pre2021":{"tau":8e-4,"eps_r":0,"eps_s":3.326e-6},
          "post2021":{"tau":8e-4,"eps_r":5.828571e-05,"eps_s":5.1e-5}},
         "D=1.5":
         {"pre2021":{"tau":5.5e-4,"eps_r":0,"eps_s":3.326e-6},
          "post2021":{"tau":5.5e-4,"eps_r":7.314285714285714e-05,"eps_s":6.4e-5}},
         "D=2":
         {"pre2021":{"tau":0.00045,"eps_r":0,"eps_s":3.326e-6},
          "post2021":{"tau":4.5e-4,"eps_r":9e-5,"eps_s":7.875e-5}},
         "D=3":
         {"pre2021":{"tau":0.000325,"eps_r":0,"eps_s":3.326e-6},
          "post2021":{"tau":3.25e-4,"eps_r":0.00011142857142857143,"eps_s":9.75e-5}}
        }



In [124]:
network2rng = {"D=1":"D1",
               "D=1.5":"D15",
               "D=2":"D2",
               "D=3":"D3"}

In [125]:
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

S = 0
n = 10

sizes = {"pre2021":16,
         "post2021":160}

tols = {"pre2021":0.5,
         "post2021":1}

for net_type in ["D=1","D=1.5","D=2","D=3"]:
    print(net_type)
    for period in ["post2021"]:#theta[net_type].keys():
        print(period)
        
        
        
        tau = theta[net_type][period]["tau"]
        eps_r = theta[net_type][period]["eps_r"]
        eps_s = theta[net_type][period]["eps_s"]
        print(tau,eps_r,eps_s)
        
        
        if net_type == "grid":
            S = 0
            n = 10
            while abs(S/n - sizes[period])>tols[period]:
                S = 0
                n = 0
                
                net_filename = "networks/grid.pkl"

                with open((net_filename), 'rb') as f1:
                    G = load(f1)


                for rng_ID in tqdm(range(1,101,1)):
                    rng_filename = "RNG\\rng_" + net_type + "_" + period+ "_" + str(rng_ID).rjust(3,"0") + ".pkl"
                    with open((rng_filename), 'rb') as f2:
                        RNG = load(f2)
                    for pop_ID in range(1,11,1):
                        pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
                        with open((pop_filename), 'rb') as f3:
                            pop = load(f3)

                        sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)

                        if sim[1]["N2N"]>0:
                            n+=1
                            S += sim[1]["Wild"] + sim[1]["N2N"]

                        sim_filename = "simulations\\sim_" + net_type+"_"+period+"_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
                        with open((sim_filename), 'wb') as f4:
                            dump(sim,f4)
                print(S/n)
                
                
                
                
        else:
            S = 0
            n = 10
            while abs(S/n - sizes[period])>tols[period]:
                S = 0
                n = 0
                for net_ID in tqdm(range(1,11,1)):
                    net_filename = "networks/grid_r_"+net_type+"_" + str(net_ID).rjust(3,"0") + ".pkl"

                    with open((net_filename), 'rb') as f1:
                        G = load(f1)

                    for rng_ID in range(1,11,1):
                        rng_filename = "RNG\\rng_" + network2rng[net_type] + "_" + period+ "_" + "network"+ str(net_ID).rjust(3,"0") + "_"+ str(rng_ID).rjust(3,"0") + ".pkl"
                        with open((rng_filename), 'rb') as f2:
                            RNG = load(f2)
                        
                        for pop_ID in range(1,11,1):
                            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
                            with open((pop_filename), 'rb') as f3:
                                pop = load(f3)

                            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
                            if sim[1]["N2N"]>0:
                                n+=1
                                S += sum(sim[1].values())
                            sim_filename = "simulations\\sim_" + network2rng[net_type]+"_"+period+"_network"+ str(net_ID).rjust(3,"0") +  "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
                            with open((sim_filename), 'wb') as f4:
                                dump(sim,f4)
                print(S/n)

            
            

D=1
post2021
0.0008 5.828571e-05 5.1e-05


100%|██████████| 10/10 [12:54<00:00, 77.40s/it]


162.414


100%|██████████| 10/10 [12:21<00:00, 74.18s/it]


161.845


  0%|          | 0/10 [00:51<?, ?it/s]


KeyboardInterrupt: 

# Pre2021 Grid

In [15]:
theta_grid_pre2021 = {"tau": 9e-4, "eps_wild": 0, "eps_seasonal" : 3.4e-7}
tau = theta_grid_pre2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [16]:
net_filename = "networks/grid.pkl"
with open((net_filename), 'rb') as f1:
        G = load(f1)
            

for rng_ID in tqdm(range(1,101,1)):
    rng_filename = "RNG\\rng_grid_pre2021_" + str(rng_ID).rjust(3,"0") + ".pkl"
    with open((rng_filename), 'rb') as f2:
        RNG = load(f2)
    
    for pop_ID in range(1,11,1):
        pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
        with open((pop_filename), 'rb') as f3:
            pop = load(f3)
        

        sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
        sim_filename = "simulations\\sim_grid_pre2021_RNG" + str(rng_ID).rjust(3,"0") + "_pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
        with open((sim_filename), 'wb') as f4:
            dump(sim,f4)

  2%|▏         | 2/100 [00:15<12:43,  7.79s/it]


KeyboardInterrupt: 

# Post2021 Grid

In [33]:
theta_grid_post2021 = {"tau": 9e-4, "eps_wild": 4e-6, "eps_seasonal" : 3.5e-6} 
tau = theta_grid_post2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [34]:
net_filename = "networks/grid.pkl"
with open((net_filename), 'rb') as f1:
        G = load(f1)
            

for rng_ID in tqdm(range(1,101,1)):
    rng_filename = "RNG\\rng_grid_post2021_" + str(rng_ID).rjust(3,"0") + ".pkl"
    with open((rng_filename), 'rb') as f2:
        RNG = load(f2)
    
    for pop_ID in range(1,11,1):
        pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
        with open((pop_filename), 'rb') as f3:
            pop = load(f3)
        

        sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
        sim_filename = "simulations\\sim_grid_post2021_RNG" + str(rng_ID).rjust(3,"0") + "_pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
        with open((sim_filename), 'wb') as f4:
            dump(sim,f4)

100%|██████████| 100/100 [2:08:53<00:00, 77.33s/it] 


# Pre2021 D1

In [35]:
theta_D1_pre2021 = {"tau": 7.1e-4, "eps_wild": 0, "eps_seasonal" : 3.4e-7}
tau = theta_D1_pre2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [36]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=1_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D1_pre2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D1_pre2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [1:13:39<00:00, 441.92s/it]


# Post2021 D1

In [37]:
theta_D1_post2021 = {"tau": 7.1e-4 , "eps_wild": 4.23e-6, "eps_seasonal" : 3.75e-6}
tau = theta_D1_post2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [38]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=1_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D1_post2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D1_post2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [2:00:37<00:00, 723.78s/it] 


# Pre2021 D1.5

In [39]:
theta_D15_pre2021 = {"tau": 4.9e-4, "eps_wild": 0, "eps_seasonal" : 3.4e-7}
tau = theta_D15_pre2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [40]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=1.5_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D15_pre2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D15_pre2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [1:20:29<00:00, 482.91s/it]


# Post2021 D1.5

In [43]:
theta_D15_post2021 = {"tau": 4.9e-4 , "eps_wild": 4.457e-6, "eps_seasonal" : 3.9e-6}
tau = theta_D15_post2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [44]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=1.5_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D15_post2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D15_post2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

  0%|          | 0/10 [10:17<?, ?it/s]


KeyboardInterrupt: 

# Pre2021 D2

In [12]:
theta_D2_pre2021 = {"tau": 5e-4, "eps_wild": 0, "eps_seasonal" : 3.4055041584399e-7}
tau = theta_D2_pre2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [13]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=2_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D2_pre2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D2_pre2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [1:16:47<00:00, 460.71s/it]


# Post2021 D2

In [14]:
theta_D2_post2021 = {"tau": 5e-4 , "eps_wild": 4.25e-6, "eps_seasonal" : 3.75e-6}
tau = theta_D2_post2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [15]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=2_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D2_post2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D2_post2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [2:06:51<00:00, 761.16s/it] 


# Pre2021 D3

In [28]:
theta_D3_pre2021 = {"tau": 3.75e-4, "eps_wild": 0, "eps_seasonal" : 3.4055041584399e-7}
tau = theta_D3_pre2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [29]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=3_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D3_pre2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D3_pre2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [1:50:37<00:00, 663.71s/it] 


# Post2021 D3

In [30]:
theta_D3_post2021 = {"tau": 3.75e-4, "eps_wild": 5e-6, "eps_seasonal" : 4e-6}
tau = theta_D3_post2021["tau"]
f = lambda I,S: (I**0.5)*(S**0.5)
T = 365
Td = 4
Tdz = 3
Tc = 3
detected_edge_weight = 0.1

In [31]:
for net_ID in tqdm(range(1,11,1)):
    net_filename = "networks/grid_r_D=3_" + str(net_ID).rjust(3,"0") + ".pkl"
    with open((net_filename), 'rb') as f1:
            G = load(f1)
            

    for rng_ID in range(1,11,1):
        rng_filename = "RNG\\rng_D3_post2021_network" + str(net_ID).rjust(3,"0") + "_" + str(rng_ID).rjust(3,"0") + ".pkl"
        with open((rng_filename), 'rb') as f2:
            RNG = load(f2)

        for pop_ID in range(1,11,1):
            pop_filename = "populations\\pop_" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((pop_filename), 'rb') as f3:
                pop = load(f3)


            sim = simulation4(G, RNG, pop, T, f, tau, Td, Tc,Tdz, detected_edge_weight)
            sim_filename = "simulations\\sim_D3_post2021_network" + str(net_ID).rjust(3,"0") + "_RNG" + str(rng_ID).rjust(3,"0") + "_pop" + str(pop_ID).rjust(3,"0") + ".pkl"
            with open((sim_filename), 'wb') as f4:
                dump(sim,f4)

100%|██████████| 10/10 [2:10:23<00:00, 782.31s/it] 
